In [22]:
import fiona
import pyproj
import copy

from shapely.geometry import shape, mapping
from shapely.ops import transform


shapeFile = 'E:/OSM/Schutzgebiete/Test/Bruennstein-DAV.shp' # epsg:31468
shapeFile31468  = 'E:/OSM/Schutzgebiete/Test/Bruennstein-DAV-31468.shp'
shapeFile4326  = 'E:/OSM/Schutzgebiete/Test/Bruennstein-DAV-4326.shp'

project1 = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:31468',proj='tmerc',lat_0=0,lon_0=12,x_0=4500000,y_0=0,ellps='bessel',towgs=[598.1,73.7,418.2,0.202,0.045,-2.455,6.7]),
    pyproj.Proj(init='epsg:4326'))

project2 = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:31468'))

features31468 = []
features4326 = []

with fiona.open(shapeFile) as input:
    schema = input.schema
    for feat in input:
        feat2 = copy.deepcopy(feat)

        poly = shape(feat['geometry'])
        poly = transform(lambda x, y, z=None: (x, y), poly)
        poly = transform(project1.transform, poly)
        feat['geometry'] = mapping(poly)
        features4326.append(feat)

        poly2 = shape(feat2['geometry'])
        poly2 = transform(lambda x, y, z=None: (x, y), poly2)
        poly2 = transform(project1.transform, poly2)
        poly2 = transform(project2.transform, poly2)
        feat2['geometry'] = mapping(poly2)
        features31468.append(feat2)

with fiona.open(shapeFile4326, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in features4326:
        out.write(f)

with fiona.open(shapeFile31468, 'w', crs={'init':'epsg:31468'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in features31468:
        out.write(f)
